In [1]:
import pickle
import pandas as pd

ratings = pickle.load( open( "saveAdultsM.p", "rb" ) )


In [2]:
ratingsPD = pd.DataFrame(ratings, columns=['user', 'item', 'rating'] )
ratingsPD.head()

,user,item,rating
0,0,17,3.0
1,0,29,153.0
2,0,42,7.0
3,0,46,518.0
4,0,47,5.0


In [3]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = preprocessing.MinMaxScaler([1,5])
def fx(x):
    data = preprocessing.scale(x['rating'].to_numpy().reshape(-1, 1))
    scaler.fit(data)
    data=scaler.transform(data)
    x['rating']=data
    return x
ratingsPD = ratingsPD.groupby('user').apply(fx)

In [4]:
ratingsPD1= ratingsPD.groupby('user').filter(lambda x : len(x)>25)
ratingsPD2 = ratingsPD1.groupby('item').filter(lambda x : len(x)>3)
ratings = ratingsPD2

In [5]:
len(ratings)

17536885

In [6]:
ratings['item'].nunique()

329929

In [7]:
ratings['user'].nunique()

25916

In [8]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias

rm = 0

for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.3)):
    pred = als.BiasedMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))


0
0.32415701201124353
4
0.32288803601231647
0
0.32691094246580094
8
0.3291718558697475
0
0.3281606355177006
Average RMSE: 0.32625769637536184


In [9]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias

for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.3)):
    pred = als.ImplicitMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))


KeyboardInterrupt: 